In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import decomposition
from sklearn import metrics
from sklearn.cluster import KMeans
# from yellowbrick.cluster import KElbowVisualizer
# from sklearn.mixture import GaussianMixture

df_cluster = df_user_click[['devtype', 'brotype', 'log_duration', 'flg', 'domhash', 'siteid']]

scaler = StandardScaler()
minmax = MinMaxScaler()

data_scaled = scaler.fit_transform(df_cluster)
data_scaled = pd.DataFrame(data=data_scaled, columns=df_cluster.columns, index=df_cluster.index)

data_scaled = minmax.fit_transform(data_scaled)
data_scaled = pd.DataFrame(data=data_scaled, columns=df_cluster.columns, index=df_cluster.index)

In [ ]:
pca = decomposition.PCA(n_components=3)
X_pca_ratio_3d = pd.DataFrame(pca.fit_transform(data_scaled), columns=(["col1", "col2", "col3"]))

x =X_pca_ratio_3d["col1"]
y =X_pca_ratio_3d["col2"]
z =X_pca_ratio_3d["col3"]

#To plot
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.scatter(x,y,z, marker="o", cmap='rainbow', alpha=0.7, edgecolors='k')
ax.set_title("A 3D Projection Of Data In The Reduced Dimension")

pca = decomposition.PCA(n_components=2)
X_pca_ratio_2d = pd.DataFrame(pca.fit_transform(data_scaled), columns=(["col1", "col2"]))

x = X_pca_ratio_2d["col1"]
y = X_pca_ratio_2d["col2"]

ax = fig.add_subplot(1, 2, 1)
ax.scatter(x,y, cmap='rainbow', alpha=0.7, edgecolors='k')
ax.set_title("A 2D Projection Of Data In The Reduced Dimension")
plt.show()